In [ ]:
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="BZTREFbhM5T8N26Kzvy3")
project = rf.workspace("elec-g62if").project("my-first-project-ktpdl")
version = project.version(2)
dataset = version.download("yolov8")  

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Fire-Detection-10 in yolov8:: 100%|██████████| 54104/54104 [00:10<00:00, 4962.61it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [1]:
!pip install optuna

In [ ]:
import optuna
from ultralytics import YOLO

def objective(trial):
    # Hyperparameters to search
    params = {
        "epochs": trial.suggest_int("epochs", 5, 100),
        "batch": trial.suggest_categorical("batch", [8, 16, 32]),
        
        # Augmentation
        "flipud": trial.suggest_float("flipud", 0.0, 0.5),
        "fliplr": trial.suggest_float("fliplr", 0.0, 0.5),
        "mosaic": trial.suggest_float("mosaic", 0.5, 1.0),
        "mixup": trial.suggest_float("mixup", 0.0, 0.3),
        "degrees": trial.suggest_float("degrees", 0.0, 20.0),
        "scale": trial.suggest_float("scale", 0.3, 0.7),
        "shear": trial.suggest_float("shear", 0.0, 5.0),
        "hsv_h": trial.suggest_float("hsv_h", 0.0, 0.2),
        "hsv_s": trial.suggest_float("hsv_s", 0.0, 0.5),
        "hsv_v": trial.suggest_float("hsv_v", 0.0, 0.5),
        "translate": trial.suggest_float("translate", 0.0, 0.2),

        # Optimizer & LR
        "lr0": trial.suggest_float("lr0", 1e-4, 1e-2, log=True),
        "lrf": trial.suggest_float("lrf", 0.01, 0.5),
        "momentum": trial.suggest_float("momentum", 0.8, 0.98),
        "weight_decay": trial.suggest_float("weight_decay", 0.0001, 0.001),
        "warmup_epochs": trial.suggest_int("warmup_epochs", 0, 5),
        "warmup_momentum": trial.suggest_float("warmup_momentum", 0.5, 0.95),
        "warmup_bias_lr": trial.suggest_float("warmup_bias_lr", 0.05, 0.5),

        # Loss
        "box": trial.suggest_float("box", 0.1, 0.7),
        "cls": trial.suggest_float("cls", 0.3, 0.8),
        "dfl": trial.suggest_float("dfl", 1.0, 2.5),

        # Regularization
        "dropout": trial.suggest_float("dropout", 0.0, 0.5),
        "patience": trial.suggest_int("patience", 5, 20),
        "close_mosaic": trial.suggest_int("close_mosaic", 0, 15),
    }

    # Load model
    model = YOLO('yolov8n.pt')

    # Train model with the sampled hyperparameters
    results = model.train(
        data='/content/Fire-Detection-10/data.yaml',
        **params,
        verbose=False
    )

    # Evaluate the model and return mAP@0.5 as the objective metric
    try:
        metrics = model.val()
        return metrics.box.map50  # or map for overall mAP
    except:
        return 0.0  # fallback in case of errors



Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/Fire-Detection-10/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /content/Fire-Detection-10/train/labels.cache... 26203 images, 855 backgrounds, 0 corrupt: 100%|██████████| 26203/26203 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 38, len(boxes) = 43889. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/Fire-Detection-10/valid/labels.cache... 592 images, 8 backgrounds, 0 corrupt: 100%|██████████| 592/592 [00:00<?, ?it/s]

...

Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1       3.8G      1.877      2.015      1.777         24        640: 100%|██████████| 1638/1638 [10:21<00:00,  2.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:15<00:00,  1.21it/s]


                   all        592        982      0.878      0.287      0.337       0.15

1 epochs completed in 0.179 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 22.5MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:13<00:00,  1.38it/s]


                   all        592        982      0.878      0.287      0.337       0.15
                  Fire         11         17          1          0     0.0347     0.0156
                  fire        573        965      0.756      0.574       0.64      0.285
Speed: 0.3ms preprocess, 5.0ms inference, 0.0ms loss, 5.9ms postprocess per image
Results saved to runs/detect/train3


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("Best trial:")
trial = study.best_trial
print(f"mAP@0.5: {trial.value}")
print("Params:")
for key, value in trial.params.items():
    print(f"  {key}: {value}")

In [ ]:
import pandas as pd
df = study.trials_dataframe()
df.to_csv("optuna_yolov8_trials.csv", index=False)